In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

import $ivy.$                                  

import org.apache.spark.sql._

In [3]:
import org.apache.log4j.{Level,Logger}
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j.{Level,Logger}


## Create the sparkSession

In [6]:

val spark = {
            NotebookSparkSession.builder()
            .master("local[*]")
            .getOrCreate()
            }

Getting spark JARs
Creating SparkSession


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@5edd8045

## GetData and parse it 

In [7]:
val parsedData = spark.read
                    .option("header","true")
                    .option("inferSchema","true")
                    .option("nullValue","?")
                    .csv("../linkages/")

csv at cmd6.sc:5

1 / 1

csv at cmd6.sc:5

5 / 5

parsedData: DataFrame = [id_1: string, id_2: string ... 10 more fields]

In [9]:
parsedData.take(10)

take at cmd8.sc:1

1 / 1

res8: Array[Row] = Array(
  [3148,8326,1,null,1,null,1,1,1,1,1,true],
  [14055,94934,1,null,1,null,1,1,1,1,1,true],
  [33948,34740,1,null,1,null,1,1,1,1,1,true],
  [946,71870,1,null,1,null,1,1,1,1,1,true],
  [64880,71676,1,null,1,null,1,1,1,1,1,true],
  [25739,45991,1,null,1,null,1,1,1,1,1,true],
  [62415,93584,1,null,1,null,1,1,1,1,0,true],
  [27995,31399,1,null,1,null,1,1,1,1,1,true],
  [4909,12238,1,null,1,null,1,1,1,1,1,true],
  [15161,16743,1,null,1,null,1,1,1,1,1,true]
)

In [10]:
parsedData.printSchema

root
 |-- id_1: string (nullable = true)
 |-- id_2: string (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: string (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: string (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)

